<a href="https://colab.research.google.com/github/SLTHRN/Heart_disease_ML/blob/main/Project_v_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### IMPORT LIBRARIES

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import sklearn


In [ ]:
### IMPORTING CSV FILE FROM GOOGLE DRIVE

import csv
from google.colab import drive
drive.mount('/content/drive')

#FILE LOCATION IN GOOGLE DRIVE
directory = '/content/drive/MyDrive/APS360PROJECT/New data.csv'

In [3]:
### DEFINING CNN MODEL NETWORK

import torch
import torch.nn as nn
import torch.nn.functional as F

#DEFINING A 3 LAYER FULLY CONNECTED MODEL
class neuralNetwork(nn.Module):

  def __init__(self):
    super(neuralNetwork, self).__init__()
    self.layer1 = nn.Linear(13,50)
    self.layer2 = nn.Linear(50,70)
    self.layer3 = nn.Linear(70,5)

  def forward(self, x):
    #x = x.squeeze()
    activation1 = self.layer1(x)
    activation1 = F.softmax(activation1)
    activation2 = F.softmax(self.layer2(activation1))
    output = self.layer3(activation2)
    return output

In [4]:
### LOADING CSV FILE FROM DIRECTORY

def load_csv(directory):
  file = open(directory, "r")
  retrieved = csv.reader(file)

  data_size = -1
  dictionary = {}
  data = []

  for row in retrieved:
    data_size+= 1
    if data_size == 0:
      for i in range(len(row)):
        dictionary[i] = row[i]
    else:
      data.append(row)

  return data_size, data, dictionary

In [5]:
### DATA STRING TO FLOAT

def str_to_flt(data):
  fail_count = 0

  for i in range(len(data)):
    row = data[i]
    new_row = []
    for j in range(len(row)):
      if row[j].isnumeric():
        new_row.append(float(row[j]))
      else:
        test = row[j].split('.')
        if len(test) == 2 and (test[0].isnumeric() and test[1].isnumeric()):
          new_row.append(float(row[j]))
        else:
          fail_count += 1
          print('!!!!!! Not Numeric, count:' + str(fail_count))
          new_row.append(row[j])
    data[i] = new_row
  return data

In [6]:
### GET LABELS

def seperate_labels(data):
  labels = []

  for row in data:
    labels.append(int(row[-1]))
    del row[-1]

  #labels = get_one_hot(labels)

  return data, labels

In [7]:
### TURN LABELS INTO ONE-HOT ENCODINGS

def get_one_hot(labels):
  new_labels = []
  for label in labels:
    if label == 0.0:
      new_labels.append([1,0,0,0,0])
    elif label == 1.0:
      new_labels.append([0,1,0,0,0])
    elif label == 2.0:
      new_labels.append([0,0,1,0,0])
    elif label == 3.0:
      new_labels.append([0,0,0,1,0])
    elif label == 4.0:
      new_labels.append([0,0,0,0,1])
  return new_labels


In [8]:
### GENERATING SAMPLERS FOR DATALOADERS

def get_samplers(data_size):
  torch.manual_seed(0)
  index = torch.randperm(data_size)
  end_training = int(len(index)*0.6) # Defining index for last data sample in training data
  end_validation = end_training + int(len(index)*0.2) # Defining index for last data sample in validation set

  training_sampler, validation_sampler, testing_sampler = index[:end_training], index[end_training:end_validation], index[end_validation:]

  return training_sampler, validation_sampler, testing_sampler

In [9]:
### GENERATING DATA LOADERS

def get_loaders(data, sampler, batch_size = 64):
  dataset = torch.tensor(data)

  loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=sampler)
  
  return loader

In [24]:
### CALCULATE ACCURACY OF MODEL

def get_accuracy(model, d_loader, l_loader):
  correct = 0
  total = 0

  for (data, label) in zip(d_loader, l_loader):
    output = model(data)
    pred = output.max(1)[1]
    for i in range(len(pred)):
      total += 1.0
      if label[i] == pred[i]:
        correct += 1.0
  return correct/total

In [48]:
### MODEL TRAINING

def training(t_loader, t_label_loader, v_loader, v_label_loader, epochs=10, learning_rate=0.001):
  n = 0
  iters, losses, training_accuracy, validation_accuracy = [], [], [], []

  model = neuralNetwork()
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  for epoch in range(epochs):
    for (data, label) in zip(t_loader, t_label_loader):
      output = model(data)
      loss = criterion(output, label)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      iters.append(n)
      losses.append(float(loss)/batch_size)
      training_accuracy.append(get_accuracy(model, t_loader, t_label_loader))
      validation_accuracy.append(get_accuracy(model, v_loader, v_label_loader))
      n += 1

  plt.title("Training")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Accuracies")
  plt.plot(iters, validation_accuracy, label = "Validation")
  plt.plot(iters, training_accuracy, label = "Training")
  plt.xlabel("Iterations")
  plt.ylabel("Accuracy")
  plt.show

  print("Final loss: {}".format(losses[-1]))
  print("Final training accuracy: {}".format(training_accuracy[-1]))
  print("Final validation accuracy: {}".format(validation_accuracy[-1]))

In [ ]:
### RUNNING THE CODE

data_size, data, dictionary = load_csv(directory)
data = str_to_flt(data)
data, labels = seperate_labels(data)
training_sampler, validation_sampler, testing_sampler = get_samplers(data_size)

batch_size = 10
training_loader = get_loaders(data, training_sampler, batch_size)
training_label_loader = get_loaders(labels, training_sampler, batch_size)
validation_loader = get_loaders(data, validation_sampler, batch_size)
validation_label_loader = get_loaders(labels, validation_sampler, batch_size)
testing_loader = get_loaders(data, testing_sampler, batch_size)
testing_label_loader = get_loaders(labels, testing_sampler, batch_size)

training(training_loader, training_label_loader, validation_loader, validation_label_loader)
#train(training_loader, validation_loader, training_label_loader, validation_label_loader, epochs = 50, learning_rate = 0.001)